<a href="https://colab.research.google.com/github/ThousandAI/pycs4001/blob/main/class10/kaggle_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
train_data = pd.read_csv("https://raw.githubusercontent.com/ThousandAI/pycs4001/main/class09/train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/ThousandAI/pycs4001/main/class09/test.csv")
test_ids = test_data["PassengerId"]

In [2]:
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)

    data.Embarked.fillna("U", inplace=True)

    return data

In [3]:
train_data = clean(train_data)
test_data = clean(test_data)
X = train_data.iloc[:, 1:]
Y = train_data.iloc[:, 0]

In [4]:
concat_data = pd.concat([X, test_data], axis=0)
dummy_data = pd.get_dummies(concat_data)

dummy_train_data = dummy_data.iloc[:891, :]
dummy_test_data = dummy_data.iloc[891:, :]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_train_data)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [9]:
depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

100%|██████████| 48/48 [00:57<00:00,  1.20s/it]


In [10]:
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

15


In [11]:
estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=22)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  estimators.append(scores.mean())

100%|██████████| 150/150 [03:49<00:00,  1.53s/it]


In [12]:
final_estimators = np.argmax(np.array(estimators)) + 50
print(final_estimators)

107


In [13]:
final_rf = RandomForestClassifier(n_estimators=92, max_depth=22)

In [14]:
final_rf.fit(sc_x, Y)

RandomForestClassifier(max_depth=22, n_estimators=92)

In [15]:
sc_test_data_x = scaler.transform(dummy_test_data)

In [16]:
submission = final_rf.predict(sc_test_data_x)

In [17]:
print(submission)

[0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 1]


In [18]:
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission })
submission_data.to_csv("submission.csv", index=False)